In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from functions import *
from pipeline import ProcessingPipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor


raw

In [2]:
credit_card_balance = pd.read_csv('dseb-63-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_credit_card_balance.csv')
credit_card_balance.head()

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,SK_ID_CURR
0,2582071,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.0,2250.0,...,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0,87788
1,2582071,-82,16809.210,67500,0.0,0.0,0.0,0.0,3375.0,9000.0,...,16809.210,0.0,0,0.0,0.0,18.0,Active,0,0,87788
2,2582071,-84,27577.890,67500,0.0,0.0,0.0,0.0,3375.0,4500.0,...,27577.890,0.0,0,0.0,0.0,16.0,Active,0,0,87788
3,2582071,-7,65159.235,45000,0.0,0.0,0.0,0.0,2250.0,2250.0,...,65609.235,0.0,0,0.0,0.0,63.0,Active,0,0,87788
4,2582071,-59,70475.850,67500,24750.0,24750.0,0.0,0.0,3375.0,4500.0,...,70475.850,4.0,4,0.0,0.0,41.0,Active,0,0,87788


In [3]:
credit_card_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3227965 entries, 0 to 3227964
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int64  
 1   MONTHS_BALANCE              int64  
 2   AMT_BALANCE                 float64
 3   AMT_CREDIT_LIMIT_ACTUAL     int64  
 4   AMT_DRAWINGS_ATM_CURRENT    float64
 5   AMT_DRAWINGS_CURRENT        float64
 6   AMT_DRAWINGS_OTHER_CURRENT  float64
 7   AMT_DRAWINGS_POS_CURRENT    float64
 8   AMT_INST_MIN_REGULARITY     float64
 9   AMT_PAYMENT_CURRENT         float64
 10  AMT_PAYMENT_TOTAL_CURRENT   float64
 11  AMT_RECEIVABLE_PRINCIPAL    float64
 12  AMT_RECIVABLE               float64
 13  AMT_TOTAL_RECEIVABLE        float64
 14  CNT_DRAWINGS_ATM_CURRENT    float64
 15  CNT_DRAWINGS_CURRENT        int64  
 16  CNT_DRAWINGS_OTHER_CURRENT  float64
 17  CNT_DRAWINGS_POS_CURRENT    float64
 18  CNT_INSTALMENT_MATURE_CUM   float64
 19  NAME_CONTRACT_STATUS 

check null

In [4]:
credit_card_balance.isnull().sum() / len(credit_card_balance)

SK_ID_PREV                    0.000000
MONTHS_BALANCE                0.000000
AMT_BALANCE                   0.000000
AMT_CREDIT_LIMIT_ACTUAL       0.000000
AMT_DRAWINGS_ATM_CURRENT      0.187658
AMT_DRAWINGS_CURRENT          0.000000
AMT_DRAWINGS_OTHER_CURRENT    0.187658
AMT_DRAWINGS_POS_CURRENT      0.187658
AMT_INST_MIN_REGULARITY       0.081904
AMT_PAYMENT_CURRENT           0.192100
AMT_PAYMENT_TOTAL_CURRENT     0.000000
AMT_RECEIVABLE_PRINCIPAL      0.000000
AMT_RECIVABLE                 0.000000
AMT_TOTAL_RECEIVABLE          0.000000
CNT_DRAWINGS_ATM_CURRENT      0.187658
CNT_DRAWINGS_CURRENT          0.000000
CNT_DRAWINGS_OTHER_CURRENT    0.187658
CNT_DRAWINGS_POS_CURRENT      0.187658
CNT_INSTALMENT_MATURE_CUM     0.081904
NAME_CONTRACT_STATUS          0.000000
SK_DPD                        0.000000
SK_DPD_DEF                    0.000000
SK_ID_CURR                    0.000000
dtype: float64

In [8]:
# fill na with mode
credit_card_balance = credit_card_balance.fillna(credit_card_balance.mode().iloc[0])
credit_card_balance.isnull().sum().sum()

0

eda

In [12]:
# # plot correlation matrix
# corr = credit_card_balance.select_dtypes(include=np.number).corr()
# plt.figure(figsize=(20, 20))
# sns.heatmap(corr, annot=True, fmt='.2f')
# plt.show()

In [13]:
import pandas as pd
import numpy as np

# Assuming credit_card_balance is your dataframe
# credit_card_balance = pd.read_csv('path_to_your_csv.csv') # Load your data

# Aggregated features
aggregations = {
    'AMT_BALANCE': ['mean', 'max', 'min', 'sum', 'var'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'max', 'min', 'sum', 'var'],
    'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'sum'],
    'AMT_DRAWINGS_CURRENT': ['mean', 'sum'],
    'AMT_PAYMENT_CURRENT': ['mean', 'sum'],
    'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'sum'],
    'CNT_DRAWINGS_CURRENT': ['mean', 'sum'],
    'SK_DPD': ['max', 'mean', 'sum'],
    'SK_DPD_DEF': ['max', 'mean', 'sum']
}

# Apply the aggregation functions to the dataframe
credit_card_agg = credit_card_balance.groupby('SK_ID_CURR').agg(aggregations)
credit_card_agg.columns = pd.Index(['{}_{}'.format(e[0], e[1].upper()) for e in credit_card_agg.columns.tolist()])

# Credit utilization and payment ratios
credit_card_balance['UTILIZATION_RATIO'] = credit_card_balance['AMT_BALANCE'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
credit_card_balance['PAYMENT_RATIO'] = credit_card_balance['AMT_PAYMENT_CURRENT'] / credit_card_balance['AMT_INST_MIN_REGULARITY']
credit_card_balance['PAYMENT_TO_BALANCE_RATIO'] = credit_card_balance['AMT_PAYMENT_CURRENT'] / credit_card_balance['AMT_BALANCE']

# Overlimit feature
credit_card_balance['OVERLIMIT'] = credit_card_balance['AMT_BALANCE'] > credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']

# Rolling behavioral features (assuming MONTHS_BALANCE is in ascending order)
credit_card_balance.sort_values(by=['SK_ID_CURR', 'MONTHS_BALANCE'], inplace=True)
credit_card_balance['AMT_BALANCE_ROLLING_MEAN'] = credit_card_balance.groupby('SK_ID_CURR')['AMT_BALANCE'].transform(lambda x: x.rolling(window=3).mean())
credit_card_balance['AMT_PAYMENT_ROLLING_SUM'] = credit_card_balance.groupby('SK_ID_CURR')['AMT_PAYMENT_CURRENT'].transform(lambda x: x.rolling(window=3).sum())

# Merge aggregated data with the original dataframe (if needed)
credit_card_balance = credit_card_balance.merge(credit_card_agg, on='SK_ID_CURR', how='left')

# Ensure to handle missing values appropriately, which may be created by the rolling function or other operations
credit_card_balance.fillna(0, inplace=True)

# At this point, the credit_card_balance dataframe has new engineered features which can be used for modeling
credit_card_balance.head()

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,CNT_DRAWINGS_ATM_CURRENT_MEAN,CNT_DRAWINGS_ATM_CURRENT_SUM,CNT_DRAWINGS_CURRENT_MEAN,CNT_DRAWINGS_CURRENT_SUM,SK_DPD_MAX,SK_DPD_MEAN,SK_DPD_SUM,SK_DPD_DEF_MAX,SK_DPD_DEF_MEAN,SK_DPD_DEF_SUM
0,1745718,-8,4500.0,180000,0.0,4500.0,0.0,4500.0,0.0,0.00,...,0.0,0.0,2.625,21,0,0.0,0,0,0.0,0
1,1745718,-7,0.0,180000,0.0,2578.5,0.0,2578.5,0.0,7771.50,...,0.0,0.0,2.625,21,0,0.0,0,0,0.0,0
2,1745718,-6,0.0,180000,0.0,0.0,0.0,0.0,0.0,629.64,...,0.0,0.0,2.625,21,0,0.0,0,0,0.0,0
3,1745718,-5,0.0,180000,0.0,0.0,0.0,0.0,0.0,488.07,...,0.0,0.0,2.625,21,0,0.0,0,0,0.0,0
4,1745718,-4,0.0,180000,0.0,0.0,0.0,0.0,0.0,488.07,...,0.0,0.0,2.625,21,0,0.0,0,0,0.0,0


In [20]:
# encode categorical features
credit_card_balance = pd.get_dummies(credit_card_balance, drop_first=True)
credit_card_balance.head()

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,SK_DPD_SUM,SK_DPD_DEF_MAX,SK_DPD_DEF_MEAN,SK_DPD_DEF_SUM,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,1745718,-8,4500.0,180000,0.0,4500.0,0.0,4500.0,0.0,0.00,...,0,0,0.0,0,False,False,False,False,False,False
1,1745718,-7,0.0,180000,0.0,2578.5,0.0,2578.5,0.0,7771.50,...,0,0,0.0,0,False,False,False,False,False,False
2,1745718,-6,0.0,180000,0.0,0.0,0.0,0.0,0.0,629.64,...,0,0,0.0,0,False,False,False,False,False,False
3,1745718,-5,0.0,180000,0.0,0.0,0.0,0.0,0.0,488.07,...,0,0,0.0,0,False,False,False,False,False,False
4,1745718,-4,0.0,180000,0.0,0.0,0.0,0.0,0.0,488.07,...,0,0,0.0,0,False,False,False,False,False,False


In [29]:
app_train = pd.read_csv('processed-data/app_train.csv')
app_train.set_index('SK_ID_CURR', inplace=True)
app_train.head()

,TARGET,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Other,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Sales staff,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Self-employed
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,0,0,1197000.0,44487.0,1197000.0,0.026392,-11945,-376,-574.0,-580,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,0,900000.0,26316.0,900000.0,0.003122,-19158,-9203,-12984.0,-2568,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0,1,265851.0,11263.5,229500.0,0.031329,-14434,-3759,-4976.0,-3989,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0,2,545040.0,20547.0,450000.0,0.004849,-15957,-6018,-10110.0,-5219,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0,0,512064.0,25033.5,360000.0,0.018801,-17851,-495,-43.0,-181,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [30]:
# merge train data with installments
app_train = app_train.merge(credit_card_balance, left_index=True, right_index=True, how='left')
app_train.head()


,TARGET,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,SK_DPD_SUM,SK_DPD_DEF_MAX,SK_DPD_DEF_MEAN,SK_DPD_DEF_SUM,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,0,0,1197000.0,44487.0,1197000.0,0.026392,-11945,-376,-574.0,-580,...,0,0,0.0,0,False,False,False,False,False,False
1,1,0,900000.0,26316.0,900000.0,0.003122,-19158,-9203,-12984.0,-2568,...,0,0,0.0,0,False,False,False,False,False,False
2,0,1,265851.0,11263.5,229500.0,0.031329,-14434,-3759,-4976.0,-3989,...,0,0,0.0,0,False,False,False,False,False,False
3,0,2,545040.0,20547.0,450000.0,0.004849,-15957,-6018,-10110.0,-5219,...,0,0,0.0,0,False,False,False,False,False,False
4,0,0,512064.0,25033.5,360000.0,0.018801,-17851,-495,-43.0,-181,...,0,0,0.0,0,False,False,False,False,False,False


In [31]:
installments = pd.read_csv('processed-data/installments_agg_2.csv', index_col='SK_ID_CURR')
installments.head()

,NUM_INSTALMENT_VERSION_MEAN_MEAN,NUM_INSTALMENT_VERSION_MEAN_SUM,NUM_INSTALMENT_VERSION_MEAN_STD,NUM_INSTALMENT_VERSION_SUM_MEAN,NUM_INSTALMENT_VERSION_SUM_SUM,NUM_INSTALMENT_VERSION_SUM_STD,NUM_INSTALMENT_VERSION_STD_MEAN,NUM_INSTALMENT_VERSION_STD_SUM,NUM_INSTALMENT_VERSION_STD_STD,NUM_INSTALMENT_NUMBER_MEAN_MEAN,...,DAY_COUNT_STD_STD,SUM_LAST_180_DAYS_MEAN_MEAN,SUM_LAST_180_DAYS_MEAN_SUM,SUM_LAST_180_DAYS_MEAN_STD,SUM_LAST_180_DAYS_SUM_MEAN,SUM_LAST_180_DAYS_SUM_SUM,SUM_LAST_180_DAYS_SUM_STD,SUM_LAST_180_DAYS_STD_MEAN,SUM_LAST_180_DAYS_STD_SUM,SUM_LAST_180_DAYS_STD_STD
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0.0,1.008333,4.033333,0.683333,3.250000,13.0,2.500000,0.432918,1.731671,0.307497,4.275000,...,21.820181,35637.304125,142549.216500,19292.800471,3.353049e+05,1341219.645,4.668329e+05,34393.312986,137573.251946,21278.096586
1.0,0.686275,2.058824,0.595058,9.333333,28.0,9.018500,0.080845,0.242536,0.140028,6.088235,...,15.126739,82921.808824,248765.426471,71441.801782,1.238428e+06,3715284.375,1.340028e+06,63879.279643,191637.838929,66667.542746
3.0,1.195833,4.783333,0.953393,4.500000,18.0,3.109126,0.618685,2.474739,0.646078,5.708333,...,31.041649,31114.969760,124459.879040,21266.375714,5.454710e+05,2181884.085,9.239618e+05,21627.459023,86509.836092,6645.334683
4.0,1.000000,1.000000,NaN,6.000000,6.0,NaN,0.000000,0.000000,NaN,3.500000,...,NaN,31347.097500,31347.097500,NaN,1.880826e+05,188082.585,NaN,16751.552686,16751.552686,NaN
5.0,1.000000,1.000000,NaN,10.000000,10.0,NaN,0.000000,0.000000,NaN,5.500000,...,NaN,54012.042000,54012.042000,NaN,5.401204e+05,540120.420,NaN,29730.327522,29730.327522,NaN


In [32]:
# merge app_train with installments
app_train = app_train.merge(installments, left_index=True, right_index=True, how='left')
app_train.head()

,TARGET,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,DAY_COUNT_STD_STD,SUM_LAST_180_DAYS_MEAN_MEAN,SUM_LAST_180_DAYS_MEAN_SUM,SUM_LAST_180_DAYS_MEAN_STD,SUM_LAST_180_DAYS_SUM_MEAN,SUM_LAST_180_DAYS_SUM_SUM,SUM_LAST_180_DAYS_SUM_STD,SUM_LAST_180_DAYS_STD_MEAN,SUM_LAST_180_DAYS_STD_SUM,SUM_LAST_180_DAYS_STD_STD
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0.0,0,0,1197000.0,44487.0,1197000.0,0.026392,-11945,-376,-574.0,-580,...,21.820181,35637.304125,142549.216500,19292.800471,3.353049e+05,1341219.645,4.668329e+05,34393.312986,137573.251946,21278.096586
1.0,1,0,900000.0,26316.0,900000.0,0.003122,-19158,-9203,-12984.0,-2568,...,15.126739,82921.808824,248765.426471,71441.801782,1.238428e+06,3715284.375,1.340028e+06,63879.279643,191637.838929,66667.542746
2.0,0,1,265851.0,11263.5,229500.0,0.031329,-14434,-3759,-4976.0,-3989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,0,2,545040.0,20547.0,450000.0,0.004849,-15957,-6018,-10110.0,-5219,...,31.041649,31114.969760,124459.879040,21266.375714,5.454710e+05,2181884.085,9.239618e+05,21627.459023,86509.836092,6645.334683
4.0,0,0,512064.0,25033.5,360000.0,0.018801,-17851,-495,-43.0,-181,...,NaN,31347.097500,31347.097500,NaN,1.880826e+05,188082.585,NaN,16751.552686,16751.552686,NaN


In [33]:
# check inf values
app_train.replace([np.inf, -np.inf], np.nan, inplace=True)
app_train.isnull().sum().sum()

# fill na with 0
for col in app_train.columns:
    app_train[col].fillna(0, inplace=True)
app_train.isnull().sum().sum()

0

In [34]:
X, y = app_train.drop('TARGET', axis=1), app_train['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale data
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
logreg = LogisticRegression(class_weight='balanced', solver='newton-cholesky', C=100)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

gini(y_test, y_pred_proba)

0.4967451690935569